<a href="https://colab.research.google.com/github/rdwnhadi/LearningDataScience/blob/main/Multiclass_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df = pd.read_csv('imdb_indonesian.csv')
df = df.drop(columns=['judul_film'])

In [4]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [5]:
#one hot encoding
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [6]:
#mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [7]:
#membagi data untuk training dan data untuk testing.
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [8]:
#tokenizer,sequencing, dan padding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [9]:
#embeding dimensi 16, hidden layers, loss, dna accuracy
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [10]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 29s - loss: 1.6102 - accuracy: 0.1891 - val_loss: 1.6108 - val_accuracy: 0.1642
Epoch 2/30
26/26 - 26s - loss: 1.6067 - accuracy: 0.2226 - val_loss: 1.6113 - val_accuracy: 0.1642
Epoch 3/30
26/26 - 26s - loss: 1.5701 - accuracy: 0.3147 - val_loss: 1.6114 - val_accuracy: 0.1791
Epoch 4/30
26/26 - 26s - loss: 1.3809 - accuracy: 0.4142 - val_loss: 1.8033 - val_accuracy: 0.1841
Epoch 5/30
26/26 - 26s - loss: 1.1155 - accuracy: 0.6256 - val_loss: 3.9834 - val_accuracy: 0.2139
Epoch 6/30
26/26 - 26s - loss: 0.8561 - accuracy: 0.7338 - val_loss: 1.8848 - val_accuracy: 0.2935
Epoch 7/30
26/26 - 26s - loss: 0.3710 - accuracy: 0.9129 - val_loss: 2.4855 - val_accuracy: 0.2786
Epoch 8/30
26/26 - 27s - loss: 0.1331 - accuracy: 0.9739 - val_loss: 2.8773 - val_accuracy: 0.2537
Epoch 9/30
26/26 - 26s - loss: 0.0595 - accuracy: 0.9888 - val_loss: 3.3171 - val_accuracy: 0.2537
Epoch 10/30
26/26 - 27s - loss: 0.0206 - accuracy: 0.9975 - val_loss: 4.1473 - val_accuracy: 0.2537
Epoch 11/